In [ ]:
from flask import Flask  
app = Flask(__name__)    

@app.route('/')   
def main():
    """Say hello"""
    return 'Hello, world!'

if __name__ == '__main__':  
    app.run()  

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
